In [1]:
import numpy as np

In [2]:
from lightexperiments.light import Light
import tabulate
from pyearth import Earth
from sklearn.cross_validation import cross_val_score, KFold, ShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor
from itertools import product
import pandas as pd
from sklearn.utils import shuffle
#import pydot
from IPython.display import SVG
from IPython.display import display, HTML


In [3]:
light = Light()

In [4]:
light.launch()

In [5]:
exp = []
for e  in light.db.find({"tags": ["model_hyperparameters"]}):
    exp.append(e)
light.close()

In [6]:
df = pd.DataFrame(exp)
df = df[df["name"]=="NADE"]
df = df[df["dataset"]=="digits"]
df = df.dropna(axis=1)#drop col with na
df

,_id,batch_size,dataset,experiment_id,learning_rate,max_epochs,momentum,name,nb_layers,nb_units,score,tags
300,55fb3b5ac7b09a5244bd562d,128,digits,5f951ec6-5d7d-11e5-8555-b8ca3af5a6d7,0.000665,200,0.80,NADE,3,275,27.200502,[model_hyperparameters]
301,55fb3b5ac7b09a5244bd562e,128,digits,5f951ec6-5d7d-11e5-8555-b8ca3af5a6d7,0.000374,200,0.80,NADE,2,162,27.183205,[model_hyperparameters]
302,55fb3b5ac7b09a5244bd562f,128,digits,5f951ec6-5d7d-11e5-8555-b8ca3af5a6d7,0.000362,200,0.50,NADE,2,112,27.204748,[model_hyperparameters]
303,55fb3b5ac7b09a5244bd5630,128,digits,5f951ec6-5d7d-11e5-8555-b8ca3af5a6d7,0.000060,200,0.80,NADE,1,213,27.184121,[model_hyperparameters]
304,55fb3b5ac7b09a5244bd5631,128,digits,5f951ec6-5d7d-11e5-8555-b8ca3af5a6d7,0.000078,200,0.50,NADE,4,131,27.247030,[model_hyperparameters]
305,55fb3b5ac7b09a5244bd5632,128,digits,5f951ec6-5d7d-11e5-8555-b8ca3af5a6d7,0.000027,200,0.90,NADE,4,248,27.210336,[model_hyperparameters]
306,55fb3b5ac7b09a5244bd5633,128,digits,5f951ec6-5d7d-11e5-8555-b8ca3af5a6d7,0.000012,200,0.90,NADE,1,180,27.255485,[model_hyperparameters]
307,55fb3b5ac7b09a5244bd5634,128,digits,5f951ec6-5d7d-11e5-8555-b8ca3af5a6d7,0.000530,200,0.90,NADE,3,198,27.188036,[model_hyperparameters]
308,55fb3b5ac7b09a5244bd5635,128,digits,5f951ec6-5d7d-11e5-8555-b8ca3af5a6d7,0.000128,200,0.99,NADE,4,277,27.196244,[model_hyperparameters]
309,55fb3b5ac7b09a5244bd5636,128,digits,5f951ec6-5d7d-11e5-8555-b8ca3af5a6d7,0.000339,200,0.90,NADE,2,202,27.208016,[model_hyperparameters]


In [20]:
labels = ["batch_size", 
          #"latent_dim",
          "learning_rate", 
          "momentum", 
          "nb_units",#nade
          "nb_layers",#nade
        #  "nb_layers_decoder", # va
        #  "nb_layers_encoder", # va
        #  "nb_units_encoder",  # va
        #  "nb_units_decoder"   # va
]
X = df[labels].values
y = df[["score"]].values
y = y[:, 0]
print(X.shape, y.shape)
print(y)

(100, 5) (100,)
[ 27.20050235  27.1832049   27.20474816  27.18412146  27.24703035
  27.21033584  27.25548462  27.18803611  27.19624361  27.20801612
  27.20500067  27.21120209  27.19808087  27.19607306  27.20859658
  27.18166228  27.18376715  27.20991401  27.19834385  27.21789802
  27.19631802  27.18785283  27.22870627  27.20002114  27.19052643
  27.20409073  27.2194152   27.19546126  27.18705766  27.18106971
  27.20246508  27.16103614  27.17882936  27.20015001  27.223667
  27.19526501  27.20814254  27.18729217  27.22395826  27.19443601
  27.21004807  27.19055718  27.19820918  27.19264673  27.1901641
  27.18347562  27.20030501  27.21919784  27.20091292  27.20918373
  27.19188279  27.24646021  27.22615788  27.20514308  27.21277807
  27.22100027  27.18499146  27.19341929  27.208734    27.18618658
  27.20638991  27.18267969  27.19532021  27.22003258  27.16356973
  27.20900813  27.22400984  27.20630376  27.18988198  27.20957874
  27.18643273  27.19076501  27.20930367  27.18200881  27.173557

In [21]:
#model = LinearRegression()
model = Earth(max_terms=40,
              thresh=0,
              max_degree=20)
#model = RandomForestRegressor(n_estimators=300, max_depth=1)
from sklearn.gaussian_process import GaussianProcess
scores = cross_val_score(model, 
                         X, 
                         y, 
                        # fit_params=dict(linvars=labels, xlabels=labels),
                         cv=5, 
                         scoring=lambda est, X, y:((est.predict(X)-y)**2).mean())
print(scores)
print(scores.mean(), scores.std())

[ 0.00039794  0.00026805  0.00027688  0.00027342  0.00033143]
0.000309544811604 4.97697810397e-05


In [22]:
%%time
models = []
mse=[]
for train, test in KFold(X.shape[0], n_folds=5):
    model = Earth(max_terms=10, 
                  thresh=0.001, 
                  check_every=1,
                  penalty=10,
                  enable_pruning=True,
                  #smooth=True,
                  max_degree=5)
    #model = RandomForestRegressor(n_estimators=300, max_depth=10)
    #model.fit(X[train], y[train])
    model.fit(X[train], y[train],
              #linvars=labels,
    #          xlabels=labels
    )
    #pqrint(model.summary())
    m_train=((np.abs(model.predict(X[train]) - y[train])).mean())
    m_test=((np.abs(model.predict(X[test]) - y[test])).mean())
    print("train : {0}, test : {1}".format(m_train, m_test))
    mse.append(m_test)
    models.append(model)
mse = np.array(mse)
print(mse)
print(mse.mean(), mse.std())

train : 0.0131413422344679, test : 0.013799522774664296
train : 0.01352431775689249, test : 0.012694283985179489
train : 0.013425434304411964, test : 0.01266315449488804
train : 0.013085213859374934, test : 0.014604755857546437
train : 0.013375646883348224, test : 0.014112574378946797
[ 0.01379952  0.01269428  0.01266315  0.01460476  0.01411257]
0.0135748582982 0.000775489089408
CPU times: user 192 ms, sys: 128 ms, total: 320 ms
Wall time: 196 ms


In [23]:
def draw(models, thresh=0.8):
    graph = pydot.Dot(graph_type='graph')

    nodes = dict()
    for label in labels:
        nodes[label] = pydot.Node(label)
    nodes_used = set()
    
    for model in models:
        for bf in model.basis_:
            nodes_used |= set([labels[v] for v in bf.variables()])
    for node in nodes_used:
        graph.add_node(nodes[node])

    all_edges = dict()
    for model in models:
        edges = dict()

        for bf in model.basis_:
            names = [labels[v] for v in bf.variables()]
            for n1, n2 in product(names, repeat=2):
                if n1 == n2:
                    continue
                if n1 < n2:
                    n1, n2 = n2, n1
                if (n1, n2) not in edges:
                    edges[(n1, n2)] = 1                    
                else:
                    edges[(n1, n2)] += 1
        for n1, n2 in edges.keys():
            if (n1, n2) in all_edges:
                all_edges[(n1,  n2)] += 1
            else:
                all_edges[(n1, n2)] = 1
    
    for (n1, n2), nb in all_edges.items():
        ratio =  float(nb) / len(models)
        if ratio >= thresh:
            edge = pydot.Edge(nodes[n1], nodes[n2])
            graph.add_edge(edge)
    graph.write_svg('a.svg')
    return SVG("a.svg") 

In [24]:
draw(models, thresh=0.6)

NameError: name 'pydot' is not defined